# Code snippets no longer needed

#### From "train_relabeled.csv" and "valid_relabeled.csv"

In [ ]:
# # Load label data from "train_relabeled.csv" and "valid_relabeled.csv"

# # Limit scope to 8 diseases in original paper
# columns_to_keep = ['Path', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']

# # Load "train_relabeled.csv" and "valid_relabeled.csv"
# train_labels_df = pd.read_csv(google_drive_path + 'label_data/train_relabeled.csv')[columns_to_keep]
# valid_labels_df = pd.read_csv(google_drive_path + 'label_data/valid_relabeled.csv')[columns_to_keep]

# # Examine first 5 rows of training dataframe
# print(f'Shape of train_labels_df: {train_labels_df.shape}')
# display(train_labels_df.head())

# # Examine first 5 rows of validation dataframe
# print(f'Shape of valid_labels_df: {valid_labels_df.shape}')
# display(valid_labels_df.head())

# # Stack the two DataFrames
# labels_df = pd.concat([train_labels_df, valid_labels_df], ignore_index=True, axis=0)

# # Remove the 'images/' prefix from the Path column
# labels_df['Path'] = labels_df['Path'].apply(lambda x: re.sub('^(.*[/])', '', x))

# # Examine first 5 rows of combined dataframe
# print(f'Shape of labels_df: {labels_df.shape}')
# display(labels_df.head())

In [ ]:
# Function to partition the TF datasets into Training and Validation sets

def partition_tf_data(ds, ds_size, train_split=0.8, val_split=0.2, 
                      test_split=0, shuffle=True, shuffle_size=10000):
  
    # assert(train_split + test_split + val_split) == 1
    assert(train_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=109)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    # test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds #, test_ds

In [ ]:
# Partition the TF datasets

DS_SIZE = images_ds.cardinality().numpy()

train_ds, val_ds = partition_tf_data(ds=complete_ds, ds_size=DS_SIZE)

print("Train shape: ", train_ds)
print("Validation shape: ", val_ds)
print("number of training images/labels: ", train_ds.cardinality().numpy())
print("number of validation images/labels: ", val_ds.cardinality().numpy())

In [ ]:
# Select images for train, validation, and test sets

# Get image names
train_selected_images = [item for item in selected_images_filenames if item in train_image_names]
val_selected_images = [item for item in selected_images_filenames if item in val_image_names]
test_selected_images = [item for item in selected_images_filenames if item in test_image_names]

print('Number of training images:', len(train_selected_images))
print('Number of validation images:', len(val_selected_images))
print('Number of testing images:', len(test_selected_images))

train_selected_images[:5]

In [ ]:
# Create full paths for selected images

train_selected_images_full_path = [selected_images_path + x for x in train_selected_images]
val_selected_images_full_path = [selected_images_path + x for x in val_selected_images]
test_selected_images_full_path = [selected_images_path + x for x in test_selected_images]

In [ ]:
# Function to get image filenames

def get_filenames(path, subdir):
  # Path to images
  selected_images_path = path + subdir
  # Filenames with full absolute paths
  selected_images_filenames_full_path = glob(selected_images_path + '/*')
  # Sort to establish consistent order
  selected_images_filenames_full_path.sort()
  # Get terminal filenames from full path
  selected_images_filenames = [re.sub('^(.*[/])', '', x) for x in selected_images_filenames_full_path]

  return selected_images_path, selected_images_filenames, selected_images_filenames_full_path

In [ ]:
# Get image filenames

filenames_out = get_filenames(path=base_path+'images_rescaled/', subdir='images_001/')
selected_images_path, selected_images_filenames, selected_images_filenames_full_path = filenames_out

In [ ]:
# # Function to downsample images filenames by keeping first n filenames

# def downsample_image_filenames(df, keep):
#   """
#   keep = number to downsample each image class to (if disease occurence were independent)
#   Returns a list of terminal image filesnames without base path
#   """
#   # List of dieases to downsample with frequency greater than keep
#   diseases_downsample = df.drop(columns='image_filename') \
#                           .columns[df.drop(columns='image_filename') \
#                           .sum(axis=0) > keep].tolist()
#   # List of diseases to retain full sample                          
#   diseases_fullsample = list(set(diseases).difference(set(diseases_downsample)))   
#   # Get first n=keep image filenames for downsampled diseases
#   downsample_list = [df['image_filename'][df[disease] == 1][:keep] for disease in diseases_downsample]
#   # Get all image filenames for full sample diseases
#   fullsample_list = [df['image_filename'][df[disease] == 1] for disease in diseases_fullsample]
#   # Flatten lists
#   downsample_list_flat = flatten_list(downsample_list) 
#   fullsample_list_flat = flatten_list(fullsample_list)    
#   # Get unique set of combined downsampled and fullsample image labels
#   filenames_to_keep = list(set(downsample_list_flat + fullsample_list_flat))
#   return filenames_to_keep

There are a total of 14 unique diseases, plus a 'no finding' class that indicates the absense of evidence for disease being present. There are also 836 different combinations of diseases.

The majority of patients (\~54%) have no evidence of disease, while \~28% have been diagnosed with a single disease, and \~19% have been diagnosed with multiple diseases.

Most diseases had some degree of correlation with one another, indicating co-morbidity, though the magnitude of correlation is quite low. The highest correlations occur between 'pneumothorax' and 'emphysema' (r=0.178), between 'pneumonia' and 'edema' (r=0.174), and between 'effusion' and 'atelectasis' (r=0.172).

Most diseases had some degree of correlation with one another, indicating co-morbidity, though the magnitude of correlation is quite low. The highest correlations occur between 'pneumothorax' and 'emphysema' (r=0.178), between 'pneumonia' and 'edema' (r=0.174), and between 'effusion' and 'atelectasis' (r=0.172).

Occurence differs dramatically among different diseases. Several diseases (e.g., hernia, pneumonia, fibrosis) have only a few hundred occurences in the X-ray images, while others (e.g., infiltration, effusion, atelectasis) have over 10,000 occurences. This means that the data exhibit extreme class imbalance.

### Test on one directory of images

In [ ]:
# # Select images for train, validation, and test sets

# # Get image filenames from directory
# selected_images_filenames_fullpath = get_filenames(path=base_path+'images_rescaled/', subdir='images_006/')

# # Get terminal filenames from full path
# selected_images_filenames = [re.sub('^(.*[/])', '', x) for x in selected_images_filenames_fullpath]

# # Get image names
# train_selected_images = [item for item in selected_images_filenames if item in train_image_names]
# val_selected_images = [item for item in selected_images_filenames if item in val_image_names]
# test_selected_images = [item for item in selected_images_filenames if item in test_image_names]

# selected_images_path = base_path + 'images_rescaled/images_006/'

# train_selected_images_fullpath = [selected_images_path + x for x in train_selected_images]
# val_selected_images_fullpath = [selected_images_path + x for x in val_selected_images]
# test_selected_images_fullpath = [selected_images_path + x for x in test_selected_images]

# print('Number of training images:', len(train_selected_images_fullpath))
# print('Number of validation images:', len(val_selected_images_fullpath))
# print('Number of testing images:', len(test_selected_images_fullpath))

# train_selected_images_fullpath[:5]

In [ ]:
# # Train label class frequencies

# train_labels_df = one_hot_disease_df[one_hot_disease_df['image_filename'].isin(train_selected_images)]
# print(f"Train labels class frequencies: \n{train_labels_df.drop(columns='image_filename').sum(axis=0)}")